In [0]:
# Importing Libraries

In [0]:
import pandas as pd
import numpy as np

In [0]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [0]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [0]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/My Drive/HAR/UCI_HAR_Dataset/{subset}/Inertial_Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [0]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/content/drive/My Drive/HAR/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [0]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [13]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [0]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [15]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [0]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [0]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [0]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [19]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [20]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [0]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 92s 13ms/step - loss: 1.3018 - acc: 0.4395 - val_loss: 1.1254 - val_acc: 0.4662
Epoch 2/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.9666 - acc: 0.5880 - val_loss: 0.9491 - val_acc: 0.5714
Epoch 3/30
7352/7352 [==============================] - 97s 13ms/step - loss: 0.7812 - acc: 0.6408 - val_loss: 0.8286 - val_acc: 0.5850
Epoch 4/30
7352/7352 [==============================] - 95s 13ms/step - loss: 0.6941 - acc: 0.6574 - val_loss: 0.7297 - val_acc: 0.6128
Epoch 5/30
7352/7352 [==============================] - 92s 13ms/step - loss: 0.6336 - acc: 0.6912 - val_loss: 0.7359 - val_acc: 0.6787
Epoch 6/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.5859 - acc: 0.7134 - val_loss: 0.7015 - val_acc: 0.6939
Epoch 7/30
7352/7352 [==============================] - 95s 13ms/step - loss: 0.5692 - acc: 0.7477 - val_loss: 0.5995 - val_acc: 0.7387


In [0]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 512        0        25        0                   0   
SITTING                  3      410        75        0                   0   
STANDING                 0       87       445        0                   0   
WALKING                  0        0         0      481                   2   
WALKING_DOWNSTAIRS       0        0         0        0                 382   
WALKING_UPSTAIRS         0        0         0        2                  18   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            3  
STANDING                           0  
WALKING                           13  
WALKING_DOWNSTAIRS                38  
WALKING_UPSTAIRS                 451  


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 4s 2ms/step


In [0]:
score

[0.3087582236972612, 0.9097387173396675]

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

In [50]:
fig , ax = plt.subplots(1,1)

<IPython.core.display.Javascript object>

In [0]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import time
# https://gist.github.com/greydanus/f6eee59eaf1d90fcb3b534a25362cea4
# https://stackoverflow.com/a/14434334
# this function is used to update the plots for each epoch and error
def plt_dynamic_plot(x, vy, ty, ax, colors=['b']):
    fig , ax = plt.subplots(1,1)
    ax.plot(x, vy, 'b', label="Validation Loss")
    ax.plot(x, ty, 'r', label="Train Loss")
    plt.legend()
    plt.grid()
    fig.canvas.draw()

In [0]:
# Initiliazing the sequential model

def create_LSTM_model(dropout = 0.1, optimizer ='rmsprop',n_hidden = 32):

    model = Sequential()
    # Configuring the parameters
    model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
    # Adding a dropout layer
    model.add(Dropout(dropout))
    # Adding a dense output layer with sigmoid activation
    model.add(Dense(n_classes, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    return model

In [0]:
from sklearn.model_selection import GridSearchCV
from keras.layers import Dense, Input, Dropout
from keras import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

model_hp = KerasClassifier(build_fn = create_LSTM_model, epochs=30, batch_size=16)

In [21]:
# define the grid search parameters
dropout = [0.3,0.4,0.5,0.6]
n_hidden = [35,40,52,56]


param_grid = dict( dropout = dropout, n_hidden = n_hidden)

model_grid = GridSearchCV(estimator=model_hp, param_grid=param_grid, n_jobs=-1, cv=3)
model_result = model_grid.fit(X_train,Y_train, batch_size=batch_size,validation_data=(X_test, Y_test), epochs=epochs)

# summarize results
print("Best: %f using %s" % (model_result.best_score_, model_result.best_params_))

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/30




7352/7352 [==============================] - 36s 5ms/step - loss: 1.2580 - acc: 0.4491 - val_loss: 1.2065 - val_acc: 0.4483
Epoch 2/30
7352/7352 [==============================] - 37s 5ms/step - loss: 1.0177 - acc: 0.5400 - val_loss: 1.1445 - val_acc: 0.4951
Epoch 3/30
7352/7352 [==============================] - 35s 5ms/step - loss: 0.7743 - acc: 0.6473 - val_loss: 0.7985 - val_acc: 0.6162
Epoch 4/30
7352/7352 [==============================] - 36s 5ms/step - loss: 0.6574 - acc: 0.7148 - val_loss: 0.6960 - val_acc: 0.7068
Epoch 5/30
7352/7352 [==============================] - 35s 5ms/step - loss: 0.5237 - acc: 0.8164 - val_loss: 0.6128 - val_acc: 0.8419
Epoch 6/30
7352/7352 [==============================] - 3

### Run Train vs Test data on Best parameters

In [19]:
model_tuned = Sequential()
# Configuring the parameters
model_tuned.add(LSTM(52, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model_tuned.add(Dropout(0.4))
# Adding a dense output layer with sigmoid activation
model_tuned.add(Dense(n_classes, activation='sigmoid'))




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [20]:
model_tuned.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

history_tuned = model_tuned.fit(X_train, Y_train,
          batch_size=16,
          epochs=30,
          verbose=1,
          validation_data=(X_test, Y_test))
score_tuned = model_tuned.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score_tuned[0])
print('Test accuracy:', score_tuned[1])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/30




7352/7352 [==============================] - 38s 5ms/step - loss: 1.2325 - acc: 0.4581 - val_loss: 1.1053 - val_acc: 0.4941
Epoch 2/30
7352/7352 [==============================] - 37s 5ms/step - loss: 1.0265 - acc: 0.5284 - val_loss: 1.0074 - val_acc: 0.5786
Epoch 3/30
7352/7352 [==============================] - 37s 5ms/step - loss: 0.7745 - acc: 0.6387 - val_loss: 0.8072 - val_acc: 0.5938
Epoch 4/30
7352/7352 [==============================] - 36s 5ms/step - loss: 0.6556 - acc: 0.7067 - val_loss: 0.7486 - val_acc: 0.6960
Epoch 5/30
7352/7352 [==============================] - 37s 5ms/step - loss: 0.5630 - acc: 0.7745 - val_loss: 0.6779 - val_acc: 0.7486
Epoch 6/30
7352/7352 [==============================] - 37s 5ms/step - loss: 0.4793 - acc: 0.8060 - val_loss: 0.5241 - val_acc: 0.7869
Epoch 7/30
7352/7352 [==================

In [2]:
pip install mpld3

     |████████████████████████████████| 798kB 2.7MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=c516cd230eec43a41dc0786c1400334ac90b3e603a0ab74aaff1f48d541387c3
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
Successfully built mpld3


In [26]:
import mpld3
from mpld3 import plugins

score_tuned = model_tuned.evaluate( X_test,Y_test, verbose=0)

print('Test score:', score_tuned[0]) 
print('Test accuracy:', score_tuned[1])

ax.set_xlabel('epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')

# list of epoch numbers
x_task1 = list(range(1,epochs+1))

vy_task1 = history_tuned.history['val_loss']
ty_task1 = history_tuned.history['loss']

print(vy_task1)
print(ty_task1)

plt_dynamic_plot(x_task1, vy_task1, ty_task1, ax)
mpld3.display()


Test score: 0.3882474149415083
Test accuracy: 0.9195792331184255
[1.1052516780093358, 1.007400548926826, 0.8072054080126607, 0.7485919309584699, 0.6778704413735224, 0.5241064832188393, 0.4650080417936101, 0.5164894072315553, 0.43848061577891306, 0.36728225362527905, 0.3482145374461637, 0.5497299465828398, 0.3411842788092275, 0.25906522794065157, 0.2986735072996452, 0.5174608449404123, 0.2983435132493801, 0.3482147149562622, 0.31072190842678293, 0.3882514895702614, 0.3401883760806517, 0.36255835237409495, 0.3376941309104325, 0.4575649724266741, 0.44280103576733826, 0.34100390364740557, 0.4976037994088862, 0.3633469000526815, 0.2531304164422931, 0.3882474233401916]
[1.2324611805987435, 1.0265450283025632, 0.7744624197677638, 0.65559397623251, 0.5630432712045925, 0.4792538316063056, 0.38396258797330357, 0.2915415139108799, 0.25903813445269736, 0.23592952726423028, 0.1952862721200023, 0.17313565549129564, 0.18192019666949444, 0.1718920935591393, 0.16507040469585973, 0.16106606266913293, 0.

<IPython.core.display.Javascript object>

In [71]:
confusion_matrix(Y_test, model_tuned.predict(X_test))

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,0,368,123,0,0,0
STANDING,0,60,472,0,0,0
WALKING,0,0,0,464,25,7
WALKING_DOWNSTAIRS,0,0,0,3,402,15
WALKING_UPSTAIRS,0,2,0,0,2,467


### 2 layer LSTM with large Dropouts

In [0]:

def create_2LSTM_model(dropout = 0.1, optimizer ='rmsprop',n_hidden = 32):

    model2 = Sequential()
    # Configuring the parameters
    model2.add(LSTM(n_hidden, return_sequences=True, input_shape=(timesteps, input_dim)))
    # Adding a dropout layer
    model2.add(Dropout(dropout))
    model2.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
    model2.add(Dropout(dropout))
    # Adding a dense output layer with sigmoid activation
    model2.add(Dense(n_classes, activation='sigmoid'))
    model2.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    return model2

In [0]:
model_2layerhp = KerasClassifier(build_fn = create_2LSTM_model, epochs=30, batch_size=16)

In [49]:
dropout = [0.7,0.8,0.9]
n_hidden = [32,40,52]


param_grid = dict( dropout = dropout, n_hidden = n_hidden )

model_2layer = GridSearchCV(estimator=model_2layerhp, param_grid=param_grid, n_jobs=-1, cv= 2)
model_2layer_result = model_2layer.fit(X_train,Y_train, batch_size=batch_size,validation_data=(X_test, Y_test), epochs=epochs)

# summarize results
print("Best: %f using %s" % (model_2layer_result.best_score_, model_2layer_result.best_params_))

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 81s 11ms/step - loss: 1.3162 - acc: 0.4475 - val_loss: 1.0442 - val_acc: 0.5830
Epoch 2/30
7352/7352 [==============================] - 79s 11ms/step - loss: 0.9885 - acc: 0.5909 - val_loss: 0.8353 - val_acc: 0.6047
Epoch 3/30
7352/7352 [==============================] - 80s 11ms/step - loss: 0.8702 - acc: 0.6209 - val_loss: 0.7861 - val_acc: 0.6715
Epoch 4/30
7352/7352 [==============================] - 84s 11ms/step - loss: 0.7738 - acc: 0.6376 - val_loss: 0.7156 - val_acc: 0.6335
Epoch 5/30
7352/7352 [==============================] - 85s 12ms/step - loss: 0.7533 - acc: 0.6598 - val_loss: 0.7167 - val_acc: 0.6922
Epoch 6/30
7352/7352 [==============================] - 85s 12ms/step - loss: 0.7067 - acc: 0.6827 - val_loss: 0.7629 - val_acc: 0.6105
Epoch 7/30
7352/7352 [==============================] - 82s 11ms/step - loss: 0.6698 - acc: 0.6965 - val_loss: 0.7611 - val_acc: 0.6189


In [0]:
model_2ltuned = Sequential()
# Configuring the parameters
model_2ltuned.add(LSTM(80, return_sequences=True, kernel_initializer='glorot_uniform'))
# Adding a dropout layer
model_2ltuned.add(Dropout(0.8))
model_2ltuned.add(BatchNormalization())
model_2ltuned.add(LSTM(80, kernel_initializer='glorot_uniform'))
model_2ltuned.add(Dropout(0.8))
# Adding a dense output layer with sigmoid activation
model_2ltuned.add(Dense(n_classes, activation='softmax'))

In [45]:
model_2ltuned.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

history_2ltuned = model_2ltuned.fit(X_train, Y_train,
          batch_size=64,
          epochs=15,
          verbose=1,
          validation_data=(X_test, Y_test))
score_2ltuned = model_2ltuned.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score_2ltuned[0])
print('Test accuracy:', score_2ltuned[1])

Train on 7352 samples, validate on 2947 samples
Epoch 1/15
7352/7352 [==============================] - 45s 6ms/step - loss: 1.2259 - acc: 0.4908 - val_loss: 0.9020 - val_acc: 0.5965
Epoch 2/15
7352/7352 [==============================] - 41s 6ms/step - loss: 0.7379 - acc: 0.6945 - val_loss: 1.1543 - val_acc: 0.6474
Epoch 3/15
7352/7352 [==============================] - 41s 6ms/step - loss: 0.6200 - acc: 0.7622 - val_loss: 0.7348 - val_acc: 0.7479
Epoch 4/15
7352/7352 [==============================] - 41s 6ms/step - loss: 0.4551 - acc: 0.8555 - val_loss: 0.8013 - val_acc: 0.7313
Epoch 5/15
7352/7352 [==============================] - 41s 6ms/step - loss: 0.3104 - acc: 0.9025 - val_loss: 0.5455 - val_acc: 0.8273
Epoch 6/15
7352/7352 [==============================] - 41s 6ms/step - loss: 0.3582 - acc: 0.8882 - val_loss: 0.5165 - val_acc: 0.8100
Epoch 7/15
7352/7352 [==============================] - 41s 6ms/step - loss: 0.3394 - acc: 0.8958 - val_loss: 1.3822 - val_acc: 0.6647
Epoch 8

In [52]:
import mpld3
from mpld3 import plugins

score_2ltuned = model_2ltuned.evaluate( X_test,Y_test, verbose=0)

print('Test score:', score_2ltuned[0]) 
print('Test accuracy:', score_2ltuned[1])

ax.set_xlabel('epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')

# list of epoch numbers
x_task1 = list(range(1,15+1))

vy_task1 = history_2ltuned.history['val_loss']
ty_task1 = history_2ltuned.history['loss']

print(vy_task1)
print(ty_task1)

plt_dynamic_plot(x_task1, vy_task1, ty_task1, ax)
mpld3.display()

Test score: 0.3620372748806869
Test accuracy: 0.8846284356973193
[0.9019750010008565, 1.1543389822735315, 0.7348045028940798, 0.8012816614500134, 0.545484897131015, 0.5164960334737968, 1.382238187509882, 0.35317547844509867, 0.32963203386051887, 0.3259659292084561, 0.3057684504400075, 0.30525806530127597, 0.3328627586570912, 0.4367965132192891, 0.3620372790566273]
[1.225917573008885, 0.737897875129982, 0.6200193295476226, 0.45512418690521644, 0.3103685539227964, 0.3581659166026816, 0.33938817632859886, 0.3735972273531365, 0.2404014403334899, 0.22583643132182277, 0.18512259257533734, 0.18192267088752576, 0.1730467584369232, 0.16085848828394603, 0.16950491364230527]


<IPython.core.display.Javascript object>

In [70]:
confusion_matrix(Y_test, model_2ltuned.predict(X_test))

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,510,0,0,0,0,27
SITTING,5,420,61,0,0,5
STANDING,0,116,415,1,0,0
WALKING,0,0,0,478,6,12
WALKING_DOWNSTAIRS,0,0,0,1,419,0
WALKING_UPSTAIRS,0,0,0,22,3,446


In [0]:
from keras.layers.normalization import BatchNormalization

model3_BN = Sequential()
model3_BN.add(LSTM(32, return_sequences=True,kernel_initializer='glorot_uniform'))
model3_BN.add(Dropout(0.5))
model3_BN.add(BatchNormalization())
model3_BN.add(LSTM(64,kernel_initializer='glorot_uniform'))
model3_BN.add(Dropout(0.5))
model3_BN.add(Dense(n_classes, activation='softmax'))

In [32]:
model3_BN.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])
history_model3 = model3_BN.fit(X_train,Y_train,batch_size=64,validation_data=(X_test, Y_test),epochs=15)

Train on 7352 samples, validate on 2947 samples
Epoch 1/15
7352/7352 [==============================] - 30s 4ms/step - loss: 1.0205 - acc: 0.5888 - val_loss: 0.8543 - val_acc: 0.6620
Epoch 2/15
7352/7352 [==============================] - 27s 4ms/step - loss: 0.4121 - acc: 0.8630 - val_loss: 0.6059 - val_acc: 0.7889
Epoch 3/15
7352/7352 [==============================] - 27s 4ms/step - loss: 0.2570 - acc: 0.9165 - val_loss: 0.3788 - val_acc: 0.8758
Epoch 4/15
7352/7352 [==============================] - 27s 4ms/step - loss: 0.1973 - acc: 0.9328 - val_loss: 0.3123 - val_acc: 0.8931
Epoch 5/15
7352/7352 [==============================] - 27s 4ms/step - loss: 0.1766 - acc: 0.9346 - val_loss: 0.6621 - val_acc: 0.8222
Epoch 6/15
7352/7352 [==============================] - 27s 4ms/step - loss: 0.1907 - acc: 0.9327 - val_loss: 0.3763 - val_acc: 0.8795
Epoch 7/15
7352/7352 [==============================] - 27s 4ms/step - loss: 0.1641 - acc: 0.9431 - val_loss: 0.4959 - val_acc: 0.8582
Epoch 8

In [33]:
score_3BN = model3_BN.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score_3BN[0])
print('Test accuracy:', score_3BN[1])

Test loss: 0.2801610008103335
Test accuracy: 0.9212758737699356


In [245]:
score_3BN = model3_BN.evaluate( X_test,Y_test, verbose=0)

print('Test score:', score_3BN[0]) 
print('Test accuracy:', score_3BN[1])

ax.set_xlabel('epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')

# list of epoch numbers
x_task1 = list(range(1,15+1))

vy_task1 = history_model3.history['val_loss']
ty_task1 = history_model3.history['loss']

print(vy_task1)
print(ty_task1)

plt_dynamic_plot(x_task1, vy_task1, ty_task1, ax)
mpld3.display()

Test score: 0.28016100497819557
Test accuracy: 0.9212758737699356
[0.8543283758140799, 0.6059476831604919, 0.37879919630867526, 0.312258508172699, 0.6621226031931441, 0.37627733805544566, 0.49591911638005154, 0.4006004456980081, 0.3460746547072136, 0.2911300434518263, 0.2784883492619297, 0.41140616562724375, 0.4062874037279815, 0.35461832352162254, 0.28016100111103026]
[1.0204617922012902, 0.41205343347898116, 0.256961944151236, 0.19729011605561622, 0.17664646338651177, 0.19072281197118812, 0.16412355951048735, 0.144158141592252, 0.16654289470857842, 0.15793967101078427, 0.1536518697575205, 0.1362282255982678, 0.1418059280018552, 0.1252058780994327, 0.14789696409892983]


<IPython.core.display.Javascript object>

In [246]:
confusion_matrix(Y_test, model3_BN.predict(X_test))

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,510,0,0,0,0,27
SITTING,5,397,74,0,0,15
STANDING,0,83,449,0,0,0
WALKING,0,1,7,482,3,3
WALKING_DOWNSTAIRS,0,0,0,0,420,0
WALKING_UPSTAIRS,0,1,0,12,1,457


## Using CNN Architecture

In [0]:
from keras.layers.convolutional import Conv1D ,MaxPooling1D

model4_BN = Sequential()
model4_BN.add(Conv1D(32,3,activation='relu',padding='valid',input_shape=(timesteps, input_dim)))
model4_BN.add(BatchNormalization())
model4_BN.add(MaxPooling1D(pool_size=2))
model4_BN.add(Conv1D(48,3, padding='valid', activation='relu'))
model4_BN.add(BatchNormalization())
model4_BN.add(MaxPooling1D(pool_size=2))
model4_BN.add(Conv1D(64,3, padding='valid', activation='relu'))
model4_BN.add(BatchNormalization())
model4_BN.add(MaxPooling1D(pool_size=2))
model4_BN.add(Conv1D(128,5,padding='valid',activation='relu'))
model4_BN.add(MaxPooling1D(pool_size=4))
model4_BN.add(Flatten())
model4_BN.add(Dense(16, activation='relu'))
model4_BN.add(Dense(n_classes, activation='softmax'))

In [242]:
model4_BN.compile(loss='categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])
history_model4 = model4_BN.fit(X_train,Y_train,batch_size=16,validation_data=(X_test, Y_test),epochs=15)

Train on 7352 samples, validate on 2947 samples
Epoch 1/15
7352/7352 [==============================] - 30s 4ms/step - loss: 0.2922 - acc: 0.8840 - val_loss: 0.2033 - val_acc: 0.9169
Epoch 2/15
7352/7352 [==============================] - 8s 1ms/step - loss: 0.1407 - acc: 0.9372 - val_loss: 0.1962 - val_acc: 0.9311
Epoch 3/15
7352/7352 [==============================] - 8s 1ms/step - loss: 0.1172 - acc: 0.9502 - val_loss: 0.2612 - val_acc: 0.9311
Epoch 4/15
7352/7352 [==============================] - 8s 1ms/step - loss: 0.1174 - acc: 0.9509 - val_loss: 0.2132 - val_acc: 0.9301
Epoch 5/15
7352/7352 [==============================] - 8s 1ms/step - loss: 0.1162 - acc: 0.9524 - val_loss: 0.2697 - val_acc: 0.9243
Epoch 6/15
7352/7352 [==============================] - 8s 1ms/step - loss: 0.1093 - acc: 0.9533 - val_loss: 0.1797 - val_acc: 0.9369
Epoch 7/15
7352/7352 [==============================] - 8s 1ms/step - loss: 0.0930 - acc: 0.9621 - val_loss: 0.4637 - val_acc: 0.9114
Epoch 8/15
73

In [243]:
score_4BN = model4_BN.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score_4BN[0])
print('Test accuracy:', score_4BN[1])

Test loss: 0.16795373193306978
Test accuracy: 0.9453681710213777


In [244]:
score_4BN = model4_BN.evaluate( X_test,Y_test, verbose=0)

print('Test score:', score_4BN[0]) 
print('Test accuracy:', score_4BN[1])

ax.set_xlabel('epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')

# list of epoch numbers
x_task1 = list(range(1,15+1))

vy_task1 = history_model4.history['val_loss']
ty_task1 = history_model4.history['loss']

print(vy_task1)
print(ty_task1)

plt_dynamic_plot(x_task1, vy_task1, ty_task1, ax)
mpld3.display()

Test score: 0.16795373193306978
Test accuracy: 0.9453681710213777
[0.20325752102651423, 0.1961597157072711, 0.261181096223429, 0.21315644821775173, 0.26972441978058664, 0.17966743743169594, 0.46369743079874226, 0.25125248902523556, 0.2662349990351221, 0.20327006207220238, 0.3494389985284612, 0.340731804485431, 0.2567077164214705, 0.31580340727535877, 0.1679537306584709]
[0.2922106668194989, 0.1406654687882102, 0.11721167276041852, 0.11739735754122664, 0.11624899146758158, 0.10925040768504853, 0.09297219137807469, 0.08526058638243313, 0.0860902275434078, 0.09170510055125114, 0.09417734731288203, 0.07913021292558732, 0.07308445791184387, 0.07572169402036656, 0.076246753867482]


<IPython.core.display.Javascript object>

In [247]:
confusion_matrix(Y_test, model4_BN.predict(X_test))

Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,7,404,76,0,0,4
STANDING,0,34,498,0,0,0
WALKING,0,0,0,480,14,2
WALKING_DOWNSTAIRS,0,0,0,0,420,0
WALKING_UPSTAIRS,0,1,0,1,22,447


### Conclusion

In [251]:
from prettytable import PrettyTable
    
x = PrettyTable()
x.field_names = ["Architecture", "Activation",  "Test score", "Test Accuracy"]


x.add_row(["1 Layer LSTM + Dropouts", "Sigmoid", 0.308, 0.9097 ])
x.add_row(["1 Layer LSTM + Dropouts", "Sigmoid", 0.388, 0.9195])
x.add_row(["2 Layer LSTM + Large Dropouts", "Sigmoid", 0.294, 0.9121])
x.add_row(["2 Layer LSTM + Dropouts + BN", "Sigmoid", 0.2801, 0.9212])
x.add_row(["4 Layer CNN + Maxpool + BN", "Relu",0.167, 0.9453])

print(x)

+-------------------------------+------------+------------+---------------+
|          Architecture         | Activation | Test score | Test Accuracy |
+-------------------------------+------------+------------+---------------+
|    1 Layer LSTM + Dropouts    |  Sigmoid   |   0.308    |     0.9097    |
|    1 Layer LSTM + Dropouts    |  Sigmoid   |   0.388    |     0.9195    |
| 2 Layer LSTM + Large Dropouts |  Sigmoid   |   0.294    |     0.9121    |
|  2 Layer LSTM + Dropouts + BN |  Sigmoid   |   0.2801   |     0.9212    |
|   4 Layer CNN + Maxpool + BN  |    Relu    |   0.167    |     0.9453    |
+-------------------------------+------------+------------+---------------+
